In [ ]:
import io
from google.colab import files
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
import random
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt

## **Lendo as bases de dados**

In [ ]:
#uploaded = files.upload()

In [ ]:
import pandas as pd
import numpy as np
BaseOriginal = pd.read_excel('satisfaction.xlsx')
BaseReduzida1 = pd.read_csv('BaseReduzida1-1.csv',delimiter=';')
BaseReduzida2 = pd.read_csv('BaseReduzida2_corrigida.csv')
BaseReduzida3 = pd.read_csv('BaseReduzida3_corrigida.csv')

In [ ]:
BaseReduzida1 = pd.read_csv('BaseReduzida1_CheckpointFinal.csv')

In [ ]:
BaseOriginal.head()

,id,satisfaction_v2,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,Inflight wifi service,Inflight entertainment,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes
0,11112,satisfied,Female,Loyal Customer,65,Personal Travel,Eco,265,0,0,0,2,2,4,2,3,3,0,3,5,3,2,0,0.0
1,110278,satisfied,Male,Loyal Customer,47,Personal Travel,Business,2464,0,0,0,3,0,2,2,3,4,4,4,2,3,2,310,305.0
2,103199,satisfied,Female,Loyal Customer,15,Personal Travel,Eco,2138,0,0,0,3,2,0,2,2,3,3,4,4,4,2,0,0.0
3,47462,satisfied,Female,Loyal Customer,60,Personal Travel,Eco,623,0,0,0,3,3,4,3,1,1,0,1,4,1,3,0,0.0
4,120011,satisfied,Female,Loyal Customer,70,Personal Travel,Eco,354,0,0,0,3,4,3,4,2,2,0,2,4,2,5,0,0.0


In [ ]:
BaseReduzida1.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,Inflight wifi service,Inflight entertainment,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction_v2
0,1,1,60,1,1,3453,5,5,5,5,5,4,5,3,3,3,3,3,3,3,3,0.0,1
1,1,1,60,1,1,2761,3,3,3,3,3,4,4,5,5,5,5,4,5,3,1,0.0,1
2,0,1,29,0,2,1374,2,4,2,1,3,2,3,3,5,5,5,3,5,3,0,0.0,0
3,0,1,39,0,2,2108,3,4,3,3,5,3,5,5,5,3,5,5,4,5,0,0.0,0
4,1,1,40,1,1,2606,1,1,1,1,3,4,4,5,5,5,5,3,5,5,11,0.0,1


In [ ]:
BaseReduzida2.head()

,Unnamed: 0,Inflight entertainment,Ease of Online booking,Online support,On-board service,Online boarding,satisfaction_v2
0,0,4,3,5,3,3,satisfied
1,1,4,5,4,5,3,satisfied
2,2,2,3,3,5,3,neutral or dissatisfied
3,3,3,5,5,5,5,neutral or dissatisfied
4,4,4,5,4,5,5,satisfied


In [ ]:
BaseReduzida3.head()

,Unnamed: 0,componente 1,componente 2,componente 3,satisfaction_v2
0,0,-1.242236,-2.611036,1.648796,satisfied
1,1,-2.317433,0.490719,-1.556811,satisfied
2,2,-0.075436,1.022683,-2.431732,neutral or dissatisfied
3,3,-2.471238,0.329657,0.083808,neutral or dissatisfied
4,4,-1.860251,3.257834,-1.006860,satisfied


In [ ]:
BaseReduzida1.shape

(14752, 25)

In [ ]:
BaseReduzida2.shape

(14709, 7)

In [ ]:
BaseReduzida3.shape

(14752, 5)

In [ ]:
BaseOriginal.shape

(129880, 24)

## **Missing values**

In [ ]:
print(BaseOriginal.isna().sum())

id                                     0
satisfaction_v2                        0
Gender                                 0
Customer Type                          0
Age                                    0
Type of Travel                         0
Class                                  0
Flight Distance                        0
Seat comfort                           0
Departure/Arrival time convenient      0
Food and drink                         0
Gate location                          0
Inflight wifi service                  0
Inflight entertainment                 0
Online support                         0
Ease of Online booking                 0
On-board service                       0
Leg room service                       0
Baggage handling                       0
Checkin service                        0
Cleanliness                            0
Online boarding                        0
Departure Delay in Minutes             0
Arrival Delay in Minutes             393
dtype: int64


In [ ]:
print(BaseReduzida1.isna().sum())

Unnamed: 0                            0
id                                    0
satisfaction_v2                       0
Gender                                0
Customer Type                         0
Age                                   0
Type of Travel                        0
Class                                 0
Flight Distance                       0
Seat comfort                          0
Departure/Arrival time convenient     0
Food and drink                        0
Gate location                         0
Inflight wifi service                 0
Inflight entertainment                0
Online support                        0
Ease of Online booking                0
On-board service                      0
Leg room service                      0
Baggage handling                      0
Checkin service                       0
Cleanliness                           0
Online boarding                       0
Departure Delay in Minutes            0
Arrival Delay in Minutes             43


In [ ]:
print(BaseReduzida3.isna().sum())

Unnamed: 0          0
componente 1       43
componente 2       43
componente 3       43
satisfaction_v2    43
dtype: int64


In [ ]:
BaseOriginal = BaseOriginal.dropna(subset=["Arrival Delay in Minutes"])
BaseReduzida1 = BaseReduzida1.dropna(subset=["Arrival Delay in Minutes"])
BaseReduzida3 = BaseReduzida3.dropna(subset=["componente 1", "componente 2", "componente 3", "satisfaction_v2"])

In [ ]:
print(BaseOriginal.isna().sum())

id                                   0
satisfaction_v2                      0
Gender                               0
Customer Type                        0
Age                                  0
Type of Travel                       0
Class                                0
Flight Distance                      0
Seat comfort                         0
Departure/Arrival time convenient    0
Food and drink                       0
Gate location                        0
Inflight wifi service                0
Inflight entertainment               0
Online support                       0
Ease of Online booking               0
On-board service                     0
Leg room service                     0
Baggage handling                     0
Checkin service                      0
Cleanliness                          0
Online boarding                      0
Departure Delay in Minutes           0
Arrival Delay in Minutes             0
dtype: int64


In [ ]:
print(BaseReduzida1.isna().sum())

Unnamed: 0                           0
id                                   0
satisfaction_v2                      0
Gender                               0
Customer Type                        0
Age                                  0
Type of Travel                       0
Class                                0
Flight Distance                      0
Seat comfort                         0
Departure/Arrival time convenient    0
Food and drink                       0
Gate location                        0
Inflight wifi service                0
Inflight entertainment               0
Online support                       0
Ease of Online booking               0
On-board service                     0
Leg room service                     0
Baggage handling                     0
Checkin service                      0
Cleanliness                          0
Online boarding                      0
Departure Delay in Minutes           0
Arrival Delay in Minutes             0
dtype: int64


In [ ]:
print(BaseReduzida3.isna().sum())

Unnamed: 0         0
componente 1       0
componente 2       0
componente 3       0
satisfaction_v2    0
dtype: int64


In [ ]:
BaseReduzida1.to_csv("BaseReduzida1.csv", sep=',', index=False)

## **Transformação de Dados**

####Base Original

In [ ]:
BaseOriginal = BaseOriginal.drop('id', axis=1)

In [ ]:
#Transformando valores nominais para binários
nominal_to_num = {"satisfaction_v2": {"satisfied": 1, "neutral or dissatisfied": 0},
                "Gender": {"Female": 1, "Male": 0},
                "Customer Type": {"Loyal Customer": 1, "disloyal Customer": 0},
                "Type of Travel": {"Business travel": 1, "Personal Travel": 0},
                "Class": {"Business": 1, "Eco": 2, "Eco Plus": 3}}

BaseOriginal = BaseOriginal.replace(nominal_to_num)

In [ ]:
#Colocando o target para a última coluna
BaseOriginal = BaseOriginal[[
       'Gender', 'Customer Type', 'Age', 'Type of Travel',
       'Class', 'Flight Distance', 'Seat comfort',
       'Departure/Arrival time convenient', 'Food and drink', 'Gate location',
       'Inflight wifi service', 'Inflight entertainment', 'Online support',
       'Ease of Online booking', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Cleanliness', 'Online boarding',
       'Departure Delay in Minutes', 'Arrival Delay in Minutes','satisfaction_v2'
    ]]
BaseOriginal.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,Inflight wifi service,Inflight entertainment,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction_v2
0,1,1,65,0,2,265,0,0,0,2,2,4,2,3,3,0,3,5,3,2,0,0.0,1
1,0,1,47,0,1,2464,0,0,0,3,0,2,2,3,4,4,4,2,3,2,310,305.0,1
2,1,1,15,0,2,2138,0,0,0,3,2,0,2,2,3,3,4,4,4,2,0,0.0,1
3,1,1,60,0,2,623,0,0,0,3,3,4,3,1,1,0,1,4,1,3,0,0.0,1
4,1,1,70,0,2,354,0,0,0,3,4,3,4,2,2,0,2,4,2,5,0,0.0,1


In [ ]:
BaseOriginal.shape

(129487, 23)

####Base Reduzida 1

In [ ]:
BaseReduzida1 = BaseReduzida1.drop('id', axis=1)
BaseReduzida1 = BaseReduzida1.drop('Unnamed: 0', axis=1)

In [ ]:
#Transformando valores nominais para binários
nominal_to_num = {"satisfaction_v2": {"satisfied": 1, "neutral or dissatisfied": 0},
                "Gender": {"Female": 1, "Male": 0},
                "Customer Type": {"Loyal Customer": 1, "disloyal Customer": 0},
                "Type of Travel": {"Business travel": 1, "Personal Travel": 0},
                "Class": {"Business": 1, "Eco": 2, "Eco Plus": 3}}
BaseReduzida1 = BaseReduzida1.replace(nominal_to_num)

In [ ]:
#Colocando o target para a última coluna
BaseReduzida1 = BaseReduzida1[[
       'Gender', 'Customer Type', 'Age', 'Type of Travel',
       'Class', 'Flight Distance', 'Seat comfort',
       'Departure/Arrival time convenient', 'Food and drink', 'Gate location',
       'Inflight wifi service', 'Inflight entertainment', 'Online support',
       'Ease of Online booking', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Cleanliness', 'Online boarding',
       'Departure Delay in Minutes', 'Arrival Delay in Minutes','satisfaction_v2'
    ]]
BaseReduzida1.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,Inflight wifi service,Inflight entertainment,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction_v2
0,1,1,60,1,1,3453,5,5,5,5,5,4,5,3,3,3,3,3,3,3,3,0.0,1
1,1,1,60,1,1,2761,3,3,3,3,3,4,4,5,5,5,5,4,5,3,1,0.0,1
2,0,1,29,0,2,1374,2,4,2,1,3,2,3,3,5,5,5,3,5,3,0,0.0,0
3,0,1,39,0,2,2108,3,4,3,3,5,3,5,5,5,3,5,5,4,5,0,0.0,0
4,1,1,40,1,1,2606,1,1,1,1,3,4,4,5,5,5,5,3,5,5,11,0.0,1


In [ ]:
BaseReduzida1.shape

(14709, 23)

####Base Reduzida 2

In [ ]:
BaseReduzida2 = BaseReduzida2.drop('Unnamed: 0', axis=1)

In [ ]:
#Transformando valores nominais para binários
enconder = LabelEncoder()
BaseReduzida2['satisfaction_v2'] = enconder.fit_transform(BaseReduzida2['satisfaction_v2'])

In [ ]:
BaseReduzida2.head()

,Inflight entertainment,Ease of Online booking,Online support,On-board service,Online boarding,satisfaction_v2
0,4,3,5,3,3,1
1,4,5,4,5,3,1
2,2,3,3,5,3,0
3,3,5,5,5,5,0
4,4,5,4,5,5,1


####Base Reduzida 3

In [ ]:
BaseReduzida3 = BaseReduzida3.drop('Unnamed: 0', axis=1)

In [ ]:
#Transformando valores nominais para binários
nominal_to_num = {"satisfaction_v2": {"satisfied": 1, "neutral or dissatisfied": 0}}

In [ ]:
BaseReduzida3 = BaseReduzida3.replace(nominal_to_num)
BaseReduzida3.head()

,componente 1,componente 2,componente 3,satisfaction_v2
0,-1.242236,-2.611036,1.648796,1
1,-2.317433,0.490719,-1.556811,1
2,-0.075436,1.022683,-2.431732,0
3,-2.471238,0.329657,0.083808,0
4,-1.860251,3.257834,-1.006860,1


## **k-NN**

Para cada base de dados, serão feitos experimentos em dois cenários: uso ou não do
escalonamento dos valores dos atributos numéricos para [0,1]. Em cada um desses contextos, o
procedimento deve ser o seguinte. Serão feitos treinamentos usando 10-fold cross validation para
diferentes valores de k (devem ser escolhidos, pelo menos, três valores diferentes de k). Aqui, o
grupo deve buscar o valor de k que produza o melhor resultado para esta base de dados.
Fazer os experimentos com e sem peso (utilizado para desempate, caso haja empate).
Após
realizar os experimentos com este método, o aluno deve buscar respostas para as seguintes
perguntas:
1. Qual foi o melhor valor de k?
2. Foi importante escalonar os valores?
3. E o peso teve algum impacto no desempenho do método?

### Cenário 1: sem escalonamento dos valores

#### Base Original

In [ ]:
X = BaseOriginal.iloc[:,:-1].values
y = BaseOriginal.iloc[:,-1].values

for i in range(10):
  seed = random.randint(900,5000)
  print("Seed = ", seed) 
  rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
  for train, test in rkf.split(X, y):
      X_train = X[train]
      X_test = X[test]
      y_train = y[train]
      y_test = y[test]
    
  #classificação para diferentes valores de K
  for K in range(10):
    K = K + 1
    knn = KNeighborsClassifier(n_neighbors=K, metric='euclidean')
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    acc = str(accuracy_score(y_test, y_pred))
    print("K = ", K, "| Acurácia: ", acc) 

Seed =  4654
K =  1 | Acurácia:  0.6394557823129252
K =  2 | Acurácia:  0.6285714285714286
K =  3 | Acurácia:  0.6435374149659864
K =  4 | Acurácia:  0.6251700680272109
K =  5 | Acurácia:  0.645578231292517
K =  6 | Acurácia:  0.6319727891156462
K =  7 | Acurácia:  0.6340136054421769
K =  8 | Acurácia:  0.6340136054421769
K =  9 | Acurácia:  0.6394557823129252
K =  10 | Acurácia:  0.6306122448979592
Seed =  3194
K =  1 | Acurácia:  0.6285714285714286
K =  2 | Acurácia:  0.6251700680272109
K =  3 | Acurácia:  0.64421768707483
K =  4 | Acurácia:  0.636734693877551
K =  5 | Acurácia:  0.6448979591836734
K =  6 | Acurácia:  0.6238095238095238
K =  7 | Acurácia:  0.636734693877551
K =  8 | Acurácia:  0.6299319727891156
K =  9 | Acurácia:  0.6312925170068027
K =  10 | Acurácia:  0.6190476190476191
Seed =  3384
K =  1 | Acurácia:  0.6074829931972789
K =  2 | Acurácia:  0.6040816326530613
K =  3 | Acurácia:  0.6265306122448979
K =  4 | Acurácia:  0.6210884353741497
K =  5 | Acurácia:  0.634013

Para os resultados acima, K = 7 obteve melhor média de acurácia = 0,709159716


In [ ]:
# Analisando os resultados com pesos >> weights='uniform'
X = BaseOriginal.iloc[:,:-1].values
y = BaseOriginal.iloc[:,-1].values

rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=1)
for train, test in rkf.split(X, y):
    X_train = X[train]
    X_test = X[test]
    y_train = y[train]
    y_test = y[test]

knn = KNeighborsClassifier(n_neighbors=7, weights='uniform', metric='euclidean')
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

acc = str(accuracy_score(y_test, y_pred))
print("Acurácia: ", acc) 

Acurácia:  0.7120018535681186


In [ ]:
# Analisando os resultados com pesos >> weights='distance'
X = BaseOriginal.iloc[:,:-1].values
y = BaseOriginal.iloc[:,-1].values

rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=1)
for train, test in rkf.split(X, y):
    X_train = X[train]
    X_test = X[test]
    y_train = y[train]
    y_test = y[test]

knn = KNeighborsClassifier(n_neighbors=7, weights='distance', metric='euclidean')
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

acc = str(accuracy_score(y_test, y_pred))
print("Acurácia: ", acc) 

Acurácia:  0.7136237256719185


####Base Reduzida 1

In [ ]:
X_br1 = BaseReduzida1.iloc[:,:-1].values
y_br1 = BaseReduzida1.iloc[:,-1].values

for i in range(10):
  seed = random.randint(900,5000)
  print("Seed = ", seed) 
  rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
  for train, test in rkf.split(X_br1, y_br1):
      X_train_br1 = X_br1[train]
      X_test_br1 = X_br1[test]
      y_train_br1 = y_br1[train]
      y_test_br1 = y_br1[test]
    
  #classificação para diferentes valores de K
  for K in range(10):
    K = K + 1
    knn = KNeighborsClassifier(n_neighbors=K, metric='euclidean')
    knn.fit(X_train_br1, y_train_br1)
    y_pred_br1 = knn.predict(X_test_br1)
    acc_br1 = str(accuracy_score(y_test_br1, y_pred_br1))
    print("K = ", K, "| Acurácia: ", acc_br1) 

Seed =  0
K =  1 | Acurácia:  0.6272108843537415
K =  2 | Acurácia:  0.6020408163265306
K =  3 | Acurácia:  0.6306122448979592
K =  4 | Acurácia:  0.6136054421768707
K =  5 | Acurácia:  0.6360544217687075
K =  6 | Acurácia:  0.6251700680272109
K =  7 | Acurácia:  0.6462585034013606
K =  8 | Acurácia:  0.6292517006802721
K =  9 | Acurácia:  0.6333333333333333
K =  10 | Acurácia:  0.6142857142857143
Seed =  1
K =  1 | Acurácia:  0.6374149659863946
K =  2 | Acurácia:  0.6068027210884354
K =  3 | Acurácia:  0.6292517006802721
K =  4 | Acurácia:  0.6231292517006802
K =  5 | Acurácia:  0.6238095238095238
K =  6 | Acurácia:  0.6204081632653061
K =  7 | Acurácia:  0.6285714285714286
K =  8 | Acurácia:  0.6115646258503401
K =  9 | Acurácia:  0.6265306122448979
K =  10 | Acurácia:  0.6163265306122448
Seed =  2
K =  1 | Acurácia:  0.6122448979591837
K =  2 | Acurácia:  0.5755102040816327
K =  3 | Acurácia:  0.6238095238095238
K =  4 | Acurácia:  0.6047619047619047
K =  5 | Acurácia:  0.6238095238

Para os resultados acima, K = 7 obteve melhor média de acurácia = 0,632993197

In [ ]:
# Analisando os resultados com pesos >> weights='uniform'
X_br1 = BaseReduzida1.iloc[:,:-1].values
y_br1 = BaseReduzida1.iloc[:,-1].values

rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=7)
for train, test in rkf.split(X_br1, y_br1):
    X_train = X_br1[train]
    X_test = X_br1[test]
    y_train = y_br1[train]
    y_test = y_br1[test]

knn = KNeighborsClassifier(n_neighbors=7, weights='uniform', metric='euclidean')
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

acc = str(accuracy_score(y_test, y_pred))
print("Acurácia: ", acc) 

Acurácia:  0.6231292517006802


In [ ]:
# Analisando os resultados com pesos >> weights='distance'
X_br1 = BaseReduzida1.iloc[:,:-1].values
y_br1 = BaseReduzida1.iloc[:,-1].values

rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=7)
for train, test in rkf.split(X_br1, y_br1):
    X_train = X_br1[train]
    X_test = X_br1[test]
    y_train = y_br1[train]
    y_test = y_br1[test]

knn = KNeighborsClassifier(n_neighbors=7, weights='distance', metric='euclidean')
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

acc = str(accuracy_score(y_test, y_pred))
print("Acurácia: ", acc) 

Acurácia:  0.6285714285714286


####Base Reduzida 2

In [ ]:
X_br2 = BaseReduzida2.iloc[:,:-1].values
y_br2 = BaseReduzida2.iloc[:,-1].values

for i in range(10):
  seed = random.randint(900,5000)
  print("Seed = ", seed) 
  rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
  for train, test in rkf.split(X_br2, y_br2):
      X_train_br2 = X_br2[train]
      X_test_br2 = X_br2[test]
      y_train_br2 = y_br2[train]
      y_test_br2 = y_br2[test]
    
  #classificação para diferentes valores de K
  for K in range(10):
    K = K + 1
    knn = KNeighborsClassifier(n_neighbors=K, metric='euclidean')
    knn.fit(X_train_br2, y_train_br2)
    y_pred_br2 = knn.predict(X_test_br2)
    acc_br2 = str(accuracy_score(y_test_br2, y_pred_br2))
    print("K = ", K, "| Acurácia: ", acc_br2) 

Seed =  0
K =  1 | Acurácia:  0.7693080012357121
K =  2 | Acurácia:  0.7705437133147977
K =  3 | Acurácia:  0.8056070435588508
K =  4 | Acurácia:  0.8149521161569354
K =  5 | Acurácia:  0.8192771084337349
K =  6 | Acurácia:  0.8192771084337349
K =  7 | Acurácia:  0.8256101328390485
K =  8 | Acurácia:  0.822829780661106
K =  9 | Acurácia:  0.8448409020698178
K =  10 | Acurácia:  0.8470033982082175
Seed =  1
K =  1 | Acurácia:  0.7747914735866543
K =  2 | Acurácia:  0.7531665122026567
K =  3 | Acurácia:  0.7813561940067965
K =  4 | Acurácia:  0.7857584182885388
K =  5 | Acurácia:  0.8066110596231079
K =  6 | Acurácia:  0.7961847389558233
K =  7 | Acurácia:  0.8106271238801359
K =  8 | Acurácia:  0.8009731232622799
K =  9 | Acurácia:  0.8105498918751931
K =  10 | Acurácia:  0.8123262279888785
Seed =  2
K =  1 | Acurácia:  0.816651220265678
K =  2 | Acurácia:  0.7839048501699104
K =  3 | Acurácia:  0.8265369168983627
K =  4 | Acurácia:  0.8443002780352178
K =  5 | Acurácia:  0.840438677788

Para os resultados acima, K = 10 obteve melhor média de acurácia = 0,845759963

####Base Reduzida 3

In [ ]:
X_br3 = BaseReduzida3.iloc[:,:-1].values
y_br3 = BaseReduzida3.iloc[:,-1].values

for i in range(10):
  seed = random.randint(900,5000)
  print("Seed = ", seed) 
  rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
  for train, test in rkf.split(X_br3, y_br3):
      X_train_br3 = X_br3[train]
      X_test_br3 = X_br3[test]
      y_train_br3 = y_br3[train]
      y_test_br3 = y_br3[test]
    
  #classificação para diferentes valores de K
  for K in range(10):
    K = K + 1
    knn = KNeighborsClassifier(n_neighbors=K, metric='euclidean')
    knn.fit(X_train_br3, y_train_br3)
    y_pred_br3 = knn.predict(X_test_br3)
    acc_br3 = str(accuracy_score(y_test_br3, y_pred_br3))
    print("K = ", K, "| Acurácia: ", acc_br3) 

Seed =  0
K =  1 | Acurácia:  0.4931787175989086
K =  2 | Acurácia:  0.4747612551159618
K =  3 | Acurácia:  0.49590723055934516
K =  4 | Acurácia:  0.4870395634379263
K =  5 | Acurácia:  0.5
K =  6 | Acurácia:  0.5061391541609823
K =  7 | Acurácia:  0.514324693042292
K =  8 | Acurácia:  0.4931787175989086
K =  9 | Acurácia:  0.5156889495225102
K =  10 | Acurácia:  0.4863574351978172
Seed =  1
K =  1 | Acurácia:  0.5177353342428377
K =  2 | Acurácia:  0.5109140518417462
K =  3 | Acurácia:  0.5204638472032742
K =  4 | Acurácia:  0.49590723055934516
K =  5 | Acurácia:  0.519781718963165
K =  6 | Acurácia:  0.5136425648021828
K =  7 | Acurácia:  0.5266030013642565
K =  8 | Acurácia:  0.5163710777626194
K =  9 | Acurácia:  0.5218281036834925
K =  10 | Acurácia:  0.529331514324693
Seed =  2
K =  1 | Acurácia:  0.5102319236016372
K =  2 | Acurácia:  0.4897680763983629
K =  3 | Acurácia:  0.5061391541609823
K =  4 | Acurácia:  0.4924965893587995
K =  5 | Acurácia:  0.5020463847203275
K =  6 | 

Para os resultados acima, K = 9 obteve melhor média de acurácia = 0,515347885

### Cenário 2: com escalonamento dos valores (normalização)

#### Base Original

In [ ]:
X = BaseOriginal.iloc[:,:-1].values
y = BaseOriginal.iloc[:,-1].values

In [ ]:
minmax = MinMaxScaler()
X = minmax.fit_transform(X)

In [ ]:
for i in range(10):
  seed = random.randint(900,5000)
  print("Seed = ", seed) 
  rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
  for train, test in rkf.split(X, y):
      X_train = X[train]
      X_test = X[test]
      y_train = y[train]
      y_test = y[test]
    
  #classificação para diferentes valores de K
  for K in range(10):
    K = K + 1
    knn = KNeighborsClassifier(n_neighbors=K, metric='euclidean')
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    acc = str(accuracy_score(y_test, y_pred))
    print("K = ", K, "| Acurácia: ", acc) 

In [ ]:
#Calculando o DP
for i in range(1,31):
  seed = random.randint(900,5000)
  rkf = RepeatedKFold(n_splits=3, n_repeats=10, random_state=seed)
  for train, test in rkf.split(X, y):
      X_train = X[train]
      X_test = X[test]
      y_train = y[train]
      y_test = y[test]
    
  #classificação para K = 7
  knn = KNeighborsClassifier(n_neighbors=7, metric='euclidean')
  knn.fit(X_train, y_train)
  y_pred = knn.predict(X_test)
  acc = str(accuracy_score(y_test, y_pred))
  print(acc)

0.9249339696955655
0.927204485426996
0.9261850702006394
0.9255595199481025
0.9274361707057134
0.9266484407580742
0.9270654742597655
0.9273434965942264
0.9268337889810482
0.9273203280663547
0.9281543950697373
0.9263935869514851
0.9256753625874612
0.9266252722302025
0.9266252722302025
0.9276678559844308
0.9274593392335851
0.9273434965942264
0.9279690468467634
0.9267411148695612
0.927111811315509
0.9252351605578981
0.9259765534497938
0.9257216996432047
0.926578935174459
0.9264399240072286
0.9272739910106111
0.9270654742597655
0.9278763727352765
0.9268337889810482


Para os resultados acima, K = 9 obteve melhor média de acurácia = 0,928830707


####Base Reduzida 1

In [ ]:
X_br1 = BaseReduzida1.iloc[:,:-1].values
y_br1 = BaseReduzida1.iloc[:,-1].values

minmax = MinMaxScaler()
X_br1 = minmax.fit_transform(X_br1)

In [ ]:
for i in range(10):
  seed = random.randint(900,5000)
  print("Seed = ", seed) 
  rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
  for train, test in rkf.split(X_br1, y_br1):
      X_train_br1 = X_br1[train]
      X_test_br1 = X_br1[test]
      y_train_br1 = y_br1[train]
      y_test_br1 = y_br1[test]
    
  #classificação para diferentes valores de K
  for K in range(10):
    K = K + 1
    knn = KNeighborsClassifier(n_neighbors=K, metric='euclidean')
    knn.fit(X_train_br1, y_train_br1)
    y_pred_br1 = knn.predict(X_test_br1)
    acc_br1 = str(accuracy_score(y_test_br1, y_pred_br1))
    print("K = ", K, "| Acurácia: ", acc_br1) 

Para os resultados acima, K = 9  obteve melhor média de acurácia = 0,914489796

####Base Reduzida 2

In [ ]:
X_br2 = BaseReduzida2.iloc[:,:-1].values
y_br2 = BaseReduzida2.iloc[:,-1].values

minmax = MinMaxScaler()
X_br2 = minmax.fit_transform(X_br2)

In [ ]:
for i in range(10):
  seed = random.randint(900,5000)
  print("Seed = ", seed) 
  rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
  for train, test in rkf.split(X_br2, y_br2):
      X_train_br2 = X_br2[train]
      X_test_br2 = X_br2[test]
      y_train_br2 = y_br2[train]
      y_test_br2 = y_br2[test]
    
  #classificação para diferentes valores de K
  for K in range(10):
    K = K + 1
    knn = KNeighborsClassifier(n_neighbors=K, metric='euclidean')
    knn.fit(X_train_br2, y_train_br2)
    y_pred_br2 = knn.predict(X_test_br2)
    acc_br2 = str(accuracy_score(y_test_br2, y_pred_br2))
    print("K = ", K, "| Acurácia: ", acc_br2) 

Para os resultados acima, K = 9 obteve melhor média de acurácia = 0,859931973

####Base Reduzida 3

In [ ]:
X_br3 = BaseReduzida3.iloc[:,:-1].values
y_br3 = BaseReduzida3.iloc[:,-1].values

minmax = MinMaxScaler()
X_br3 = minmax.fit_transform(X_br3)

In [ ]:
for i in range(10):
  seed = random.randint(900,5000)
  print("Seed = ", seed) 
  rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
  for train, test in rkf.split(X_br3, y_br3):
      X_train_br3 = X_br3[train]
      X_test_br3 = X_br3[test]
      y_train_br3 = y_br3[train]
      y_test_br3 = y_br3[test]
    
  #classificação para diferentes valores de K
  for K in range(10):
    K = K + 1
    knn = KNeighborsClassifier(n_neighbors=K, metric='euclidean')
    knn.fit(X_train_br3, y_train_br3)
    y_pred_br3 = knn.predict(X_test_br3)
    acc_br3 = str(accuracy_score(y_test_br3, y_pred_br3))
    print("K = ", K, "| Acurácia: ", acc_br3) 

Para os resultados acima, K = 9 obteve melhor média de acurácia = 0,515075034

## **Árvore de Decisão**

Para cada base de dados, serão feitos treinamentos usando 10-fold cross validation,variando-se apenas o parâmetro poda (ou seja, construir árvore com poda e sem poda). Aqui, as perguntas a serem respondidas são as seguintes:
1. A árvore (Árvore de Decisão) apresentou overfitting antes da poda?
2. Com o uso da poda é possível afirmar se a AD estava ou não com overfitting?

### Sem poda

Base Original

In [ ]:
X = BaseOriginal.iloc[:,:-1].values
y = BaseOriginal.iloc[:,-1].values

rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=1)
for train, test in rkf.split(X, y):
    X_train = X[train]
    X_test = X[test]
    y_train = y[train]
    y_test = y[test]
  
DTC = DecisionTreeClassifier()
DTC.fit(X_train, y_train) 
y_pred = DTC.predict(X_test)

acc = str(accuracy_score(y_test, y_pred))
print("Acurácia: ", acc) 

#Verifica se ocorre overfiting
test_AUC = roc_auc_score(y_test, y_pred)
print("Test AUC: ", test_AUC)

y_pred = DTC.predict(X_train)
train_AUC = roc_auc_score(y_train, y_pred) 
print("Train AUC: ", train_AUC) 

Acurácia:  0.9389867160951498
Test AUC:  0.9384333579101832
Train AUC:  1.0


Train AUC = 1.0 significa que o modelo está se ajustando perfeitamente ao treino, ou seja, está ocorrendo overfiting.

Base Reduzida 1

In [ ]:
X = BaseReduzida1.iloc[:,:-1].values
y = BaseReduzida1.iloc[:,-1].values

rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=1)
for train, test in rkf.split(X, y):
    X_train = X[train]
    X_test = X[test]
    y_train = y[train]
    y_test = y[test]
  
DTC = DecisionTreeClassifier()
DTC.fit(X_train, y_train) 
y_pred = DTC.predict(X_test)

acc = str(accuracy_score(y_test, y_pred))
print("Acurácia: ", acc) 

#Verifica se ocorre overfiting
test_AUC = roc_auc_score(y_test, y_pred)
print("Test AUC: ", test_AUC)

y_pred = DTC.predict(X_train)
train_AUC = roc_auc_score(y_train, y_pred) 
print("Train AUC: ", train_AUC) 

Acurácia:  0.9210884353741496
Test AUC:  0.9199169799498748
Train AUC:  1.0


Base Reduzida 2

In [ ]:
X = BaseReduzida2.iloc[:,:-1].values
y = BaseReduzida2.iloc[:,-1].values

rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=1)
for train, test in rkf.split(X, y):
    X_train = X[train]
    X_test = X[test]
    y_train = y[train]
    y_test = y[test]
  
DTC = DecisionTreeClassifier()
DTC.fit(X_train, y_train) 
y_pred = DTC.predict(X_test)

acc = str(accuracy_score(y_test, y_pred))
print("Acurácia: ", acc) 

#Verifica se ocorre overfiting
test_AUC = roc_auc_score(y_test, y_pred)
print("Test AUC: ", test_AUC)

y_pred = DTC.predict(X_train)
train_AUC = roc_auc_score(y_train, y_pred) 
print("Train AUC: ", train_AUC) 

Acurácia:  0.8680272108843538
Test AUC:  0.8666979949874687
Train AUC:  0.890750194489358


Base Reduzida 3

In [ ]:
X = BaseReduzida3.iloc[:,:-1].values
y = BaseReduzida3.iloc[:,-1].values

rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=1)
for train, test in rkf.split(X, y):
    X_train = X[train]
    X_test = X[test]
    y_train = y[train]
    y_test = y[test]
  
DTC = DecisionTreeClassifier()
DTC.fit(X_train, y_train) 
y_pred = DTC.predict(X_test)

acc = str(accuracy_score(y_test, y_pred))
print("Acurácia: ", acc) 

#Verifica se ocorre overfiting
test_AUC = roc_auc_score(y_test, y_pred)
print("Test AUC: ", test_AUC)

y_pred = DTC.predict(X_train)
train_AUC = roc_auc_score(y_train, y_pred) 
print("Train AUC: ", train_AUC) 

Acurácia:  0.5088676671214188
Test AUC:  0.5063518837103743
Train AUC:  1.0


### Com poda

####Base Original

In [ ]:
X = BaseOriginal.iloc[:,:-1].values
y = BaseOriginal.iloc[:,-1].values

rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=1)
for train, test in rkf.split(X, y):
    X_train = X[train]
    X_test = X[test]
    y_train = y[train]
    y_test = y[test]

# Utilizando o grid search pra buscar os melhores parâmetros pra fazer a poda

# parametros pro modelo
from sklearn.model_selection import GridSearchCV
params = {'max_leaf_nodes': list(range(2, 20)), 
          'min_samples_split': [2, 3, 4, 5, 10],
          'max_depth': [2, 5, 7]}

# chamada grid
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, verbose=1, cv=3)

#treinamento
grid_search_cv.fit(X_train, y_train)

#predição
y_pred = grid_search_cv.predict(X_test)

#avalia o modelo
acc = str(accuracy_score(y_test, y_pred))
print("Acurácia: ", acc)

Fitting 3 folds for each of 270 candidates, totalling 810 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 810 out of 810 | elapsed:  2.9min finished


Acurácia:  0.8423694779116466


In [ ]:
# ver os melhores parametros
grid_search_cv.best_params_

{'max_depth': 5, 'max_leaf_nodes': 4, 'min_samples_split': 2}

In [ ]:
#Recall do modelo
rec = str(recall_score(y_test, y_pred, average = 'macro'))
print("Recall: ", rec)

#Matriz de confusão
cm = confusion_matrix(y_test, y_pred)
print("Matriz de confusão")
print(cm)

Recall:  0.8344021701278397
Matriz de confusão
[[4379 1571]
 [ 470 6528]]


In [ ]:
#Verifica se ocorre overfiting
test_AUC = roc_auc_score(y_test, y_pred)
print("Test AUC: ", test_AUC)

y_pred = grid_search_cv.predict(X_train)
train_AUC = roc_auc_score(y_train, y_pred) 
print("Train AUC: ", train_AUC) 

Test AUC:  0.8344021701278397
Train AUC:  0.8370769059800197


####Base Reduzida 1

In [ ]:
X = BaseReduzida1.iloc[:,:-1].values
y = BaseReduzida1.iloc[:,-1].values

rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=1)
for train, test in rkf.split(X, y):
    X_train = X[train]
    X_test = X[test]
    y_train = y[train]
    y_test = y[test]

# Utilizando o grid search pra buscar os melhores parâmetros pra fazer a poda

# parametros pro modelo
from sklearn.model_selection import GridSearchCV
params = {'max_leaf_nodes': list(range(2, 20)), 
          'min_samples_split': [2, 3, 4, 5, 10],
          'max_depth': [2, 5, 7]}

# chamada grid
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, verbose=1, cv=3)

#treinamento
grid_search_cv.fit(X_train, y_train)

#predição
y_pred = grid_search_cv.predict(X_test)

#avalia o modelo
acc = str(accuracy_score(y_test, y_pred))
print("Acurácia: ", acc)

Fitting 3 folds for each of 270 candidates, totalling 810 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Acurácia:  0.8979591836734694


[Parallel(n_jobs=1)]: Done 810 out of 810 | elapsed:   17.3s finished


In [ ]:
# ver os melhores parametros
grid_search_cv.best_params_

{'max_depth': 7, 'max_leaf_nodes': 19, 'min_samples_split': 2}

In [ ]:
#Recall do modelo
rec = str(recall_score(y_test, y_pred, average = 'macro'))
print("Recall: ", rec)

#Matriz de confusão
cm = confusion_matrix(y_test, y_pred)
print("Matriz de confusão")
print(cm)

Recall:  0.8963815789473684
Matriz de confusão
[[590  82]
 [ 68 730]]


In [ ]:
#Verifica se ocorre overfiting
test_AUC = roc_auc_score(y_test, y_pred)
print("Test AUC: ", test_AUC)

y_pred = DTC.predict(X_train)
train_AUC = roc_auc_score(y_train, y_pred) 
print("Train AUC: ", train_AUC) 

Error Test:  0.8963815789473685
Error Train:  0.8931714515517644


####Base Reduzida 2

In [ ]:
X = BaseReduzida2.iloc[:,:-1].values
y = BaseReduzida2.iloc[:,-1].values

rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=1)
for train, test in rkf.split(X, y):
    X_train = X[train]
    X_test = X[test]
    y_train = y[train]
    y_test = y[test]

# Utilizando o grid search pra buscar os melhores parâmetros pra fazer a poda

# parametros pro modelo
from sklearn.model_selection import GridSearchCV
params = {'max_leaf_nodes': list(range(2, 20)), 
          'min_samples_split': [2, 3, 4, 5, 10],
          'max_depth': [2, 5, 7]}

# chamada grid
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, verbose=1, cv=3)

#treinamento
grid_search_cv.fit(X_train, y_train)

#predição
y_pred = grid_search_cv.predict(X_test)

#avalia o modelo
acc = str(accuracy_score(y_test, y_pred))
print("Acurácia: ", acc)

Fitting 3 folds for each of 270 candidates, totalling 810 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Acurácia:  0.8496598639455782


[Parallel(n_jobs=1)]: Done 810 out of 810 | elapsed:    4.8s finished


In [ ]:
# ver os melhores parametros
grid_search_cv.best_params_

{'max_depth': 7, 'max_leaf_nodes': 19, 'min_samples_split': 2}

In [ ]:
#Recall do modelo
rec = str(recall_score(y_test, y_pred, average = 'macro'))
print("Recall: ", rec)

#Matriz de confusão
cm = confusion_matrix(y_test, y_pred)
print("Matriz de confusão")
print(cm)

Recall:  0.8497807017543859
Matriz de confusão
[[572 100]
 [121 677]]


In [ ]:
#Verifica se ocorre overfiting
test_AUC = roc_auc_score(y_test, y_pred)
print("Test AUC: ", test_AUC)

y_pred = DTC.predict(X_train)
train_AUC = roc_auc_score(y_train, y_pred) 
print("Train AUC: ", train_AUC) 

Error Test:  0.8497807017543859
Error Train:  0.865297076984728


####Base Reduzida 3

In [ ]:
X = BaseReduzida3.iloc[:,:-1].values
y = BaseReduzida3.iloc[:,-1].values

rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=1)
for train, test in rkf.split(X, y):
    X_train = X[train]
    X_test = X[test]
    y_train = y[train]
    y_test = y[test]

# Utilizando o grid search pra buscar os melhores parâmetros pra fazer a poda

# parametros pro modelo
from sklearn.model_selection import GridSearchCV
params = {'max_leaf_nodes': list(range(2, 20)), 
          'min_samples_split': [2, 3, 4, 5, 10],
          'max_depth': [2, 5, 7]}

# chamada grid
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, verbose=1, cv=3)

#treinamento
grid_search_cv.fit(X_train, y_train)

#predição
y_pred = grid_search_cv.predict(X_test)

#avalia o modelo
acc = str(accuracy_score(y_test, y_pred))
print("Acurácia: ", acc)

Fitting 3 folds for each of 270 candidates, totalling 810 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Acurácia:  0.5300136425648022


[Parallel(n_jobs=1)]: Done 810 out of 810 | elapsed:   12.8s finished


In [ ]:
# ver os melhores parametros
grid_search_cv.best_params_

{'max_depth': 2, 'max_leaf_nodes': 2, 'min_samples_split': 2}

In [ ]:
#Recall do modelo
rec = str(recall_score(y_test, y_pred, average = 'macro'))
print("Recall: ", rec)

#Matriz de confusão
cm = confusion_matrix(y_test, y_pred)
print("Matriz de confusão")
print(cm)

Recall:  0.5
Matriz de confusão
[[  0 689]
 [  0 777]]


In [ ]:
#Verifica se ocorre overfiting
test_AUC = roc_auc_score(y_test, y_pred)
print("Test AUC: ", test_AUC)

y_pred = DTC.predict(X_train)
train_AUC = roc_auc_score(y_train, y_pred) 
print("Train AUC: ", train_AUC) 

Error Test:  0.5
Error Train:  0.5


## **Naive Bayes**

Os experimentos serão feitos usando 10-fold cross validation. Para cada base de dados, as
perguntas a serem respondidas para o NB são as seguintes:
1. Os meus dados numéricos estavam obedecendo uma distribuição normal?
2. Os meus dados possuem alguma relação?

####Base Original

In [ ]:
acuracia = []
for i in range(10):
  seed = random.randint(900,5000)
  
  X = BaseOriginal.iloc[:,:-1].values
  y = BaseOriginal.iloc[:,-1].values

  rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
  for train, test in rkf.split(X, y):
      X_train = X[train]
      X_test = X[test]
      y_train = y[train]
      y_test = y[test]

  nb = GaussianNB()
  nb.fit(X_train, y_train)
  y_pred = nb.predict(X_test)

  acc = accuracy_score(y_test, y_pred)
  acuracia.append(acc)
print('Acurácia:', mean(acuracia))

Acurácia: 0.8169833178869323


####Base Reduzida 1

In [ ]:
acuracia = []
for i in range(10):
  seed = random.randint(900,5000)
  
  X = BaseReduzida1.iloc[:,:-1].values
  y = BaseReduzida1.iloc[:,-1].values

  rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
  for train, test in rkf.split(X, y):
      X_train = X[train]
      X_test = X[test]
      y_train = y[train]
      y_test = y[test]

  nb = GaussianNB()
  nb.fit(X_train, y_train)
  y_pred = nb.predict(X_test)

  acc = accuracy_score(y_test, y_pred)
  acuracia.append(acc)
print('Acurácia:', mean(acuracia))

Acurácia: 0.8187755102040816


####Base Reduzida 2

In [ ]:
acuracia = []
for i in range(10):
  seed = random.randint(900,5000)
  
  X = BaseReduzida2.iloc[:,:-1].values
  y = BaseReduzida2.iloc[:,-1].values

  rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
  for train, test in rkf.split(X, y):
      X_train = X[train]
      X_test = X[test]
      y_train = y[train]
      y_test = y[test]

  nb = GaussianNB()
  nb.fit(X_train, y_train)
  y_pred = nb.predict(X_test)

  acc = accuracy_score(y_test, y_pred)
  acuracia.append(acc)
print('Acurácia:', mean(acuracia))

Acurácia: 0.7672108843537415


####Base Reduzida 3

In [ ]:
acuracia = []
for i in range(10):
  seed = random.randint(900,5000)
  
  X = BaseReduzida3.iloc[:,:-1].values
  y = BaseReduzida3.iloc[:,-1].values

  rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
  for train, test in rkf.split(X, y):
      X_train = X[train]
      X_test = X[test]
      y_train = y[train]
      y_test = y[test]

  nb = GaussianNB()
  nb.fit(X_train, y_train)
  y_pred = nb.predict(X_test)

  acc = accuracy_score(y_test, y_pred)
  acuracia.append(acc)
print('Acurácia:', mean(acuracia))

Acurácia: 0.534106412005457


## **Redes Neurais**

Para cada base de dados, serão feitos treinamentos de redes MLP usando o backpropagation
padrão (com o termo momentum fixo de 0.8), variando-se os seguintes parâmetros:

a. Quantidade máxima de iterações (ou ciclos).

b. Quantidade de neurônios intermediários (ou escondidos) da rede --- serão usadas redes com apenas uma camada intermediária.

c. Taxa de aprendizado.

P/ neurônios:
Ponto de partida = 
(num. Att + num. classes)/2 = (23+1)/2 = 12

####Base Original

In [ ]:
for taxa_aprendizado in [0.1, 0.01, 0.001]:
  for iteracoes in [100, 1000, 10000]:  
    for neuronios in [12,16,20]:    

      print("Taxa aprendizado =", taxa_aprendizado, "| Neurônios =", neuronios, "| Iteracoes =", iteracoes)

      X = BaseOriginal.iloc[:,:-1].values
      y = BaseOriginal.iloc[:,-1].values

      rkf = RepeatedKFold(n_splits=2, n_repeats=10, random_state=990)
      for train, test in rkf.split(X, y):
          X_train = X[train]
          X_test = X[test]
          y_train = y[train]
          y_test = y[test]

      minmax = MinMaxScaler()
      X_train = minmax.fit_transform(X_train)
      X_test = minmax.fit_transform(X_test)

      mlp = MLPClassifier(solver='sgd', momentum=0.8, 
                          hidden_layer_sizes=neuronios, 
                          learning_rate='constant', 
                          learning_rate_init=taxa_aprendizado,
                          max_iter=iteracoes, 
                          random_state=1)

      mlp.fit(X_train, y_train)
      y_pred = mlp.predict(X_test)

      acc = str(accuracy_score(y_test, y_pred))
      print("Acurácia:", acc)

      # cm = confusion_matrix(y_test, y_pred)
      # print("Matriz de confusão")
      # print(cm)

      #Análise do MLP
      print("Loss:", mlp.loss_)
      print ("Score treino:", mlp.score(X_train,y_train))
      print ("Score teste:",mlp.score(X_test,y_test))

      #Análise AUC
      test_AUC = roc_auc_score(y_test, y_pred)
      print("Test AUC: ", test_AUC)

      y_pred = mlp.predict(X_train)
      train_AUC = roc_auc_score(y_train, y_pred) 
      print("Train AUC: ", train_AUC) 

In [ ]:
#Para o melhor resultado (Taxa aprendizado = 0.1 | Neurônios = 16 | Iteracoes = 100), aplica-se para 10-fold cross validation

X = BaseOriginal.iloc[:,:-1].values
y = BaseOriginal.iloc[:,-1].values

rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=990)
for train, test in rkf.split(X, y):
    X_train = X[train]
    X_test = X[test]
    y_train = y[train]
    y_test = y[test]

minmax = MinMaxScaler()
X_train = minmax.fit_transform(X_train)
X_test = minmax.fit_transform(X_test)

mlp = MLPClassifier(solver='sgd', momentum=0.8, 
                    hidden_layer_sizes=16, 
                    learning_rate='constant', 
                    learning_rate_init=0.1,
                    max_iter=100, 
                    random_state=1)

mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)

#Acurácia
acc = str(accuracy_score(y_test, y_pred))
print("Acurácia:", acc)

#Matriz de Confusão
cm = confusion_matrix(y_test, y_pred)
print("Matriz de confusão")
print(cm)

#Análise do MLP
print("Loss:", mlp.loss_)
print ("Score treino:", mlp.score(X_train,y_train))
print ("Score teste:",mlp.score(X_test,y_test))

#Análise AUC
test_AUC = roc_auc_score(y_test, y_pred)
print("Test AUC: ", test_AUC)

y_pred = mlp.predict(X_train)
train_AUC = roc_auc_score(y_train, y_pred) 
print("Train AUC: ", train_AUC)

Acurácia: 0.9345844918134075
Matriz de confusão
[[5616  263]
 [ 584 6485]]
Loss: 0.14533962694976302
Score treino: 0.9389045727181459
Score teste: 0.9345844918134075
Test AUC:  0.9363251348076722
Train AUC:  0.9393268797576781


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


####Base Reduzida 1

In [ ]:
for taxa_aprendizado in [0.1, 0.01, 0.001]:
  for iteracoes in [100, 1000, 10000]:  
    for neuronios in [12,16,20]:    

      print("Taxa aprendizado =", taxa_aprendizado, "| Neurônios =", neuronios, "| Iteracoes =", iteracoes)

      X = BaseReduzida1.iloc[:,:-1].values
      y = BaseReduzida1.iloc[:,-1].values

      rkf = RepeatedKFold(n_splits=2, n_repeats=10, random_state=990)
      for train, test in rkf.split(X, y):
          X_train = X[train]
          X_test = X[test]
          y_train = y[train]
          y_test = y[test]

      minmax = MinMaxScaler()
      X_train = minmax.fit_transform(X_train)
      X_test = minmax.fit_transform(X_test)

      mlp = MLPClassifier(solver='sgd', momentum=0.8, 
                          hidden_layer_sizes=neuronios, 
                          learning_rate='constant', 
                          learning_rate_init=taxa_aprendizado,
                          max_iter=iteracoes, 
                          random_state=1)

      mlp.fit(X_train, y_train)
      y_pred = mlp.predict(X_test)

      acc = str(accuracy_score(y_test, y_pred))
      print("Acurácia:", acc)

      # cm = confusion_matrix(y_test, y_pred)
      # print("Matriz de confusão")
      # print(cm)

      #Análise do MLP
      print("Loss:", mlp.loss_)
      print ("Score treino:", mlp.score(X_train,y_train))
      print ("Score teste:",mlp.score(X_test,y_test))

      #Análise AUC
      test_AUC = roc_auc_score(y_test, y_pred)
      print("Test AUC: ", test_AUC)

      y_pred = mlp.predict(X_train)
      train_AUC = roc_auc_score(y_train, y_pred) 
      print("Train AUC: ", train_AUC) 

Taxa aprendizado = 0.1 | Neurônios = 12 | Iteracoes = 100


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Acurácia: 0.9082132172967092
Loss: 0.18190316837010595
Score treino: 0.9250849762066621
Score teste: 0.9082132172967092
Test AUC:  0.9020816818958751
Train AUC:  0.9223066713176657
Taxa aprendizado = 0.1 | Neurônios = 16 | Iteracoes = 100


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Acurácia: 0.9228991025292358
Loss: 0.15630544469371743
Score treino: 0.9367777022433719
Score teste: 0.9228991025292358
Test AUC:  0.9191315640886755
Train AUC:  0.9353649601271866
Taxa aprendizado = 0.1 | Neurônios = 20 | Iteracoes = 100


...

Acurácia: 0.812754963285287
Loss: 0.42737478236770454
Score treino: 0.8182188987083616
Score teste: 0.812754963285287
Test AUC:  0.8071245221318908
Train AUC:  0.8162786676105285
Taxa aprendizado = 0.001 | Neurônios = 12 | Iteracoes = 1000
Acurácia: 0.8802012510198531
Loss: 0.27991491088334103
Score treino: 0.887423521414004
Score teste: 0.8802012510198531
Test AUC:  0.8763524820652867
Train AUC:  0.8868681965958904
Taxa aprendizado = 0.001 | Neurônios = 16 | Iteracoes = 1000


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Acurácia: 0.8859124286102801
Loss: 0.26855155391086044
Score treino: 0.8934058463630183
Score teste: 0.8859124286102801
Test AUC:  0.8817873566805852
Train AUC:  0.8932090067221193
Taxa aprendizado = 0.001 | Neurônios = 20 | Iteracoes = 1000


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Acurácia: 0.8921675278759859
Loss: 0.2533676965396594
Score treino: 0.900475866757308
Score teste: 0.8921675278759859
Test AUC:  0.8881443141740113
Train AUC:  0.9000274278267256
Taxa aprendizado = 0.001 | Neurônios = 12 | Iteracoes = 10000
Acurácia: 0.8802012510198531
Loss: 0.27991491088334103
Score treino: 0.887423521414004
Score teste: 0.8802012510198531
Test AUC:  0.8763524820652867
Train AUC:  0.8868681965958904
Taxa aprendizado = 0.001 | Neurônios = 16 | Iteracoes = 10000
Acurácia: 0.889039978243133
Loss: 0.26009571296343903
Score treino: 0.8966689326988443
Score teste: 0.889039978243133
Test AUC:  0.8850281080901288
Train AUC:  0.8963627241727362
Taxa aprendizado = 0.001 | Neurônios = 20 | Iteracoes = 10000
Acurácia: 0.8961109600217568
Loss: 0.24576218551188958
Score treino: 0.9042828008157716
Score teste: 0.8961109600217568
Test AUC:  0.8922824631306683
Train AUC:  0.9038852919875552


Melhor resultado
Taxa aprendizado = 0,1 | Neurônios = 16 | Iteracoes = 100

Acurácia = 0,9385107270284047

In [ ]:
#Para o melhor resultado (Taxa aprendizado = 0.1 | Neurônios = 16 | Iteracoes = 100), aplica-se para 10-fold cross validation

X = BaseReduzida1.iloc[:,:-1].values
y = BaseReduzida1.iloc[:,-1].values

rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=990)
for train, test in rkf.split(X, y):
    X_train = X[train]
    X_test = X[test]
    y_train = y[train]
    y_test = y[test]

minmax = MinMaxScaler()
X_train = minmax.fit_transform(X_train)
X_test = minmax.fit_transform(X_test)

mlp = MLPClassifier(solver='sgd', momentum=0.8, 
                    hidden_layer_sizes=16, 
                    learning_rate='constant', 
                    learning_rate_init=0.1,
                    max_iter=100, 
                    random_state=1)

mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)

#Acurácia
acc = str(accuracy_score(y_test, y_pred))
print("Acurácia:", acc)

#Matriz de Confusão
cm = confusion_matrix(y_test, y_pred)
print("Matriz de confusão")
print(cm)

#Análise do MLP
print("Loss:", mlp.loss_)
print ("Score treino:", mlp.score(X_train,y_train))
print ("Score teste:",mlp.score(X_test,y_test))

#Análise AUC
test_AUC = roc_auc_score(y_test, y_pred)
print("Test AUC: ", test_AUC)

y_pred = mlp.predict(X_train)
train_AUC = roc_auc_score(y_train, y_pred) 
print("Train AUC: ", train_AUC)

Acurácia: 0.9285714285714286
Matriz de confusão
[[600  72]
 [ 33 765]]
Loss: 0.1529415117615957
Score treino: 0.9358712893723091
Score teste: 0.9285714285714286
Test AUC:  0.9257518796992482
Train AUC:  0.9337154120235929


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


####Base Reduzida 2

In [ ]:
for taxa_aprendizado in [0.1, 0.01, 0.001]:
  for iteracoes in [100, 1000, 10000]:  
    for neuronios in [12,16,20]:    

      print("Taxa aprendizado =", taxa_aprendizado, "| Neurônios =", neuronios, "| Iteracoes =", iteracoes)

      X = BaseReduzida2.iloc[:,:-1].values
      y = BaseReduzida2.iloc[:,-1].values

      rkf = RepeatedKFold(n_splits=2, n_repeats=10, random_state=990)
      for train, test in rkf.split(X, y):
          X_train = X[train]
          X_test = X[test]
          y_train = y[train]
          y_test = y[test]

      minmax = MinMaxScaler()
      X_train = minmax.fit_transform(X_train)
      X_test = minmax.fit_transform(X_test)

      mlp = MLPClassifier(solver='sgd', momentum=0.8, 
                          hidden_layer_sizes=neuronios, 
                          learning_rate='constant', 
                          learning_rate_init=taxa_aprendizado,
                          max_iter=iteracoes, 
                          random_state=1)

      mlp.fit(X_train, y_train)
      y_pred = mlp.predict(X_test)

      acc = str(accuracy_score(y_test, y_pred))
      print("Acurácia:", acc)

      # cm = confusion_matrix(y_test, y_pred)
      # print("Matriz de confusão")
      # print(cm)

      #Análise do MLP
      print("Loss:", mlp.loss_)
      print ("Score treino:", mlp.score(X_train,y_train))
      print ("Score teste:",mlp.score(X_test,y_test))

      #Análise AUC
      test_AUC = roc_auc_score(y_test, y_pred)
      print("Test AUC: ", test_AUC)

      y_pred = mlp.predict(X_train)
      train_AUC = roc_auc_score(y_train, y_pred) 
      print("Train AUC: ", train_AUC) 

Taxa aprendizado = 0.1 | Neurônios = 12 | Iteracoes = 100


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Acurácia: 0.8433505575197172
Loss: 0.35167493402645444
Score treino: 0.8606390210740993
Score teste: 0.8433505575197172
Test AUC:  0.8376383879419839
Train AUC:  0.8604378027622955
Taxa aprendizado = 0.1 | Neurônios = 16 | Iteracoes = 100


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Acurácia: 0.8553168343758499
Loss: 0.33941415081038206
Score treino: 0.868388851121686
Score teste: 0.8553168343758499
Test AUC:  0.8502771282652036
Train AUC:  0.8679827568514995
Taxa aprendizado = 0.1 | Neurônios = 20 | Iteracoes = 100


...

Acurácia: 0.7318466140875713
Loss: 0.5457982555929266
Score treino: 0.7400407885791979
Score teste: 0.7318466140875713
Test AUC:  0.7161398734992381
Train AUC:  0.7319547323544554
Taxa aprendizado = 0.001 | Neurônios = 16 | Iteracoes = 100


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Acurácia: 0.7606744628773456
Loss: 0.5026405169776976
Score treino: 0.7828687967369137
Score teste: 0.7606744628773456
Test AUC:  0.7471006482099444
Train AUC:  0.7780599512595491
Taxa aprendizado = 0.001 | Neurônios = 20 | Iteracoes = 100


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Acurácia: 0.7474843622518358
Loss: 0.5010389768031385
Score treino: 0.7718558803535011
Score teste: 0.7474843622518358
Test AUC:  0.7354821727536145
Train AUC:  0.7687660834058382
Taxa aprendizado = 0.001 | Neurônios = 12 | Iteracoes = 1000
Acurácia: 0.7945335871634485
Loss: 0.45159669942897757
Score treino: 0.7997280761386811
Score teste: 0.7945335871634485
Test AUC:  0.7893493685067231
Train AUC:  0.798852456557634
Taxa aprendizado = 0.001 | Neurônios = 16 | Iteracoes = 1000
Acurácia: 0.7882784878977427
Loss: 0.4597030274137738
Score treino: 0.7968728755948334
Score teste: 0.7882784878977427
Test AUC:  0.7805513226303405
Train AUC:  0.7957483378840954
Taxa aprendizado = 0.001 | Neurônios = 20 | Iteracoes = 1000
Acurácia: 0.8001087843350557
Loss: 0.43587437471305657
Score treino: 0.8087015635622026
Score teste: 0.8001087843350557
Test AUC:  0.7945350406036406
Train AUC:  0.8082144113553282
Taxa aprendizado = 0.001 | Neurônios = 12 | Iteracoes = 10000
Acurácia: 0.7945335871634485
Loss:

Melhor resultado
Taxa aprendizado = 0,1 | Neurônios = 16 | Iteracoes = 100

Acurácia = 0,8553168343758499

In [ ]:
#Para o melhor resultado (Taxa aprendizado = 0.1 | Neurônios = 16 | Iteracoes = 100), aplica-se para 10-fold cross validation

X = BaseReduzida2.iloc[:,:-1].values
y = BaseReduzida2.iloc[:,-1].values

rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=990)
for train, test in rkf.split(X, y):
    X_train = X[train]
    X_test = X[test]
    y_train = y[train]
    y_test = y[test]

minmax = MinMaxScaler()
X_train = minmax.fit_transform(X_train)
X_test = minmax.fit_transform(X_test)

mlp = MLPClassifier(solver='sgd', momentum=0.8, 
                    hidden_layer_sizes=16, 
                    learning_rate='constant', 
                    learning_rate_init=0.1,
                    max_iter=100, 
                    random_state=1)

mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)

#Acurácia
acc = str(accuracy_score(y_test, y_pred))
print("Acurácia:", acc)

#Matriz de Confusão
cm = confusion_matrix(y_test, y_pred)
print("Matriz de confusão")
print(cm)

#Análise do MLP
print("Loss:", mlp.loss_)
print ("Score treino:", mlp.score(X_train,y_train))
print ("Score teste:",mlp.score(X_test,y_test))

#Análise AUC
test_AUC = roc_auc_score(y_test, y_pred)
print("Test AUC: ", test_AUC)

y_pred = mlp.predict(X_train)
train_AUC = roc_auc_score(y_train, y_pred) 
print("Train AUC: ", train_AUC)

Acurácia: 0.8591836734693877
Matriz de confusão
[[546 126]
 [ 81 717]]
Loss: 0.3443178476311262
Score treino: 0.8595815393911926
Score teste: 0.8591836734693877
Test AUC:  0.8554981203007519
Train AUC:  0.8563454586459427


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


####Base Reduzida 3

In [ ]:
for taxa_aprendizado in [0.1, 0.01, 0.001]:
  for iteracoes in [100, 1000, 10000]:  
    for neuronios in [12,16,20]:    

      print("Taxa aprendizado =", taxa_aprendizado, "| Neurônios =", neuronios, "| Iteracoes =", iteracoes)

      X = BaseReduzida3.iloc[:,:-1].values
      y = BaseReduzida3.iloc[:,-1].values

      rkf = RepeatedKFold(n_splits=2, n_repeats=10, random_state=990)
      for train, test in rkf.split(X, y):
          X_train = X[train]
          X_test = X[test]
          y_train = y[train]
          y_test = y[test]

      minmax = MinMaxScaler()
      X_train = minmax.fit_transform(X_train)
      X_test = minmax.fit_transform(X_test)

      mlp = MLPClassifier(solver='sgd', momentum=0.8, 
                          hidden_layer_sizes=neuronios, 
                          learning_rate='constant', 
                          learning_rate_init=taxa_aprendizado,
                          max_iter=iteracoes, 
                          random_state=1)

      mlp.fit(X_train, y_train)
      y_pred = mlp.predict(X_test)

      acc = str(accuracy_score(y_test, y_pred))
      print("Acurácia:", acc)

      # cm = confusion_matrix(y_test, y_pred)
      # print("Matriz de confusão")
      # print(cm)

      #Análise do MLP
      print("Loss:", mlp.loss_)
      print ("Score treino:", mlp.score(X_train,y_train))
      print ("Score teste:",mlp.score(X_test,y_test))

      #Análise AUC
      test_AUC = roc_auc_score(y_test, y_pred)
      print("Test AUC: ", test_AUC)

      y_pred = mlp.predict(X_train)
      train_AUC = roc_auc_score(y_train, y_pred) 
      print("Train AUC: ", train_AUC) 

Taxa aprendizado = 0.1 | Neurônios = 12 | Iteracoes = 100
Acurácia: 0.5257057138960862
Loss: 0.6890318081040679
Score treino: 0.5357970816855312
Score teste: 0.5257057138960862
Test AUC:  0.5067516222284074
Train AUC:  0.5191872348161057
Taxa aprendizado = 0.1 | Neurônios = 16 | Iteracoes = 100
Acurácia: 0.5273421519159962
Loss: 0.6892366530299601
Score treino: 0.5383881085503887
Score teste: 0.5273421519159962
Test AUC:  0.5082111144435246
Train AUC:  0.5217812942782917
Taxa aprendizado = 0.1 | Neurônios = 20 | Iteracoes = 100
Acurácia: 0.5371607800354562
Loss: 0.6890500585034786
Score treino: 0.5357970816855312
Score teste: 0.5371607800354562
Test AUC:  0.49954233022039946
Train AUC:  0.5049389122149558
Taxa aprendizado = 0.1 | Neurônios = 12 | Iteracoes = 1000
Acurácia: 0.5257057138960862
Loss: 0.6890318081040679
Score treino: 0.5357970816855312
Score teste: 0.5257057138960862
Test AUC:  0.5067516222284074
Train AUC:  0.5191872348161057
Taxa aprendizado = 0.1 | Neurônios = 16 | Iter

Taxa aprendizado = 0,01 | Neurônios = 12 | Iteracoes = 100

Acurácia = 0,5430246829401336

In [ ]:
#Para o melhor resultado (Taxa aprendizado = 0.01 | Neurônios = 12 | Iteracoes = 100), aplica-se para 10-fold cross validation

X = BaseReduzida3.iloc[:,:-1].values
y = BaseReduzida3.iloc[:,-1].values

rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=990)
for train, test in rkf.split(X, y):
    X_train = X[train]
    X_test = X[test]
    y_train = y[train]
    y_test = y[test]

minmax = MinMaxScaler()
X_train = minmax.fit_transform(X_train)
X_test = minmax.fit_transform(X_test)

mlp = MLPClassifier(solver='sgd', momentum=0.8, 
                    hidden_layer_sizes=12, 
                    learning_rate='constant', 
                    learning_rate_init=0.01,
                    max_iter=100, 
                    random_state=1)

mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)

#Acurácia
acc = str(accuracy_score(y_test, y_pred))
print("Acurácia:", acc)

#Matriz de Confusão
cm = confusion_matrix(y_test, y_pred)
print("Matriz de confusão")
print(cm)

#Análise do MLP
print("Loss:", mlp.loss_)
print ("Score treino:", mlp.score(X_train,y_train))
print ("Score teste:",mlp.score(X_test,y_test))

#Análise AUC
test_AUC = roc_auc_score(y_test, y_pred)
print("Test AUC: ", test_AUC)

y_pred = mlp.predict(X_train)
train_AUC = roc_auc_score(y_train, y_pred) 
print("Train AUC: ", train_AUC)

Acurácia: 0.5409276944065484
Matriz de confusão
[[  0 673]
 [  0 793]]
Loss: 0.6898867058464092
Score treino: 0.5386363636363637
Score teste: 0.5409276944065484
Test AUC:  0.5
Train AUC:  0.5
